In [41]:
from tensorflow.keras.layers import Conv3D,add, Dense, Conv3DTranspose,MaxPool3D,Dropout, UpSampling3D, BatchNormalization,Activation, MaxPooling3D
from tensorflow.keras.models import Sequential
import tensorflow

In [92]:
inputs =(448,448,48,1)
num_class=1
def conv_block(x, nfilter):
    x=Conv3D(nfilter,(3,3,3), padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    x=Conv3D(nfilter,(3,3,3), padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    return x
    
def upsample_block(x, nfilter,skip_cnn):
    x = UpSampling3D(2)(x)
    print(x.shape, skip_cnn.shape)
    #x=add([x,skip_cnn])
    x=Conv3DTranspose(filters, (3,3,3), padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    x=Conv3DTranspose(filters, (3,3,3), padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    
    return x
#encoder
inputs_L=tensorflow.keras.Input(inputs)
x=Conv3D(16, 3, padding="same")(inputs_L)
skip=[]
for filters in [32,64,128]:
    x=conv_block(x,filters)
    skip.append(x)
    x = MaxPooling3D((2,2,2), padding="same")(x)
    

#decoder
i=len(skip)-1
for filters in [128,64,32]:
    x=upsample_block(x,filters,skip[i])
    
    
x=Conv3D(16,3,activation="relu", padding="same")(x)
output=Conv3D(num_class,3,activation="sigmoid", padding="same")(x)

model=tensorflow.keras.Model(inputs_L,output)


(None, 112, 112, 12, 128) (None, 112, 112, 12, 128)
(None, 224, 224, 24, 128) (None, 112, 112, 12, 128)
(None, 448, 448, 48, 64) (None, 112, 112, 12, 128)


In [93]:
model.summary()

Model: "functional_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        [(None, 448, 448, 48, 1)] 0         
_________________________________________________________________
conv3d_293 (Conv3D)          (None, 448, 448, 48, 16)  448       
_________________________________________________________________
conv3d_294 (Conv3D)          (None, 448, 448, 48, 32)  13856     
_________________________________________________________________
batch_normalization_401 (Bat (None, 448, 448, 48, 32)  128       
_________________________________________________________________
activation_396 (Activation)  (None, 448, 448, 48, 32)  0         
_________________________________________________________________
conv3d_295 (Conv3D)          (None, 448, 448, 48, 32)  27680     
_________________________________________________________________
batch_normalization_402 (Bat (None, 448, 448, 48, 32)

In [96]:
from keras import backend as K

def weighted_dice_coefficient(y_true, y_pred, axis=(-3, -2, -1), smooth=0.00001):
    """
    Weighted dice coefficient. Default axis assumes a "channels first" data structure
    :param smooth:
    :param y_true:
    :param y_pred:
    :param axis:
    :return:
    """
    return K.mean(2. * (K.sum(y_true * y_pred,
                              axis=axis) + smooth/2)/(K.sum(y_true,
                                                            axis=axis) + K.sum(y_pred,
                                                                axis=axis) + smooth))
def weighted_dice_coefficient_loss(y_true, y_pred):
    return 1-weighted_dice_coefficient(y_true, y_pred) 
model.compile(optimizer='Adam', loss=weighted_dice_coefficient_loss)